In [1]:
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt

## Functions

In [36]:
def mchlis_ment(x):
    
    return 0.65*x/(1 + x/20)

def sq_exp(x, σsq):
    
    # initialize covariance
    n = len(x)
    Σ = np.zeros([n,n])
    
    # build covariance
    for i in range(n):
        for j in range(n):
            Σ[i,j] = σsq*np.exp(-((x[i] - x[j])/3)**2)
    
    return Σ

def gls_est(X, y, Σ):
    
    # transpose regressors, nxp -> pxn
    X_T = np.transpose(X)
    
    Σ_inv = np.linalg.solve(Σ,np.identity(2))
    
#     # compute repeated term, (pxn)(nxn) -> pxn
#     Σ_invX = np.solve(Σ,X)
    
#     term1 = np.dot(X_T, Σ_inv)
    
#     assert np.shape(X_TΣ_inv) == (2,5)
#     # compute first term, (pxn)(nxp)->(pxp)
#     X_TΣ_invX = np.dot(X_TΣ_inv, X)
    
#     assert np.shape(X_TΣ_invX) == (2,2)
    
#     # invert first term, (pxp)
#     term1 = np.linalg.inv(X_TΣ_invX)
    
#     # compute second term, (pxn)(nx1)->(px1)
#     term2 = np.dot(X_TΣ_inv, y)
    
#     # compute generalized least squares estimator,(pxp)x(px1)--> (px1)
#     θ_hat = np.dot(term1, term2)
    
    return θ_hat

## Synthetic data generation

In [33]:
# set seed for reproducibility
np.random.seed(234)

## generate synthetic data
x = np.linspace(0, 10, 5).reshape(5,1)

# compute ground truth model
ζ = mchlis_ment(x)

# set ground truth variance of noise
σϵsq_tru = 0.1

# generate normal i.i.d. measurement error, ϵ_1,..., ϵ_n ~iid N(0,σϵsq_tru I_n)
ϵ = stats.norm.rvs(loc = 0, scale = np.sqrt(σϵsq_tru), size = len(x))

# generate observations
y = ζ + ϵ

## Set bounds and initial values for parameter estimation

In [34]:
## variance of discrepancy
# lower bound
σδsq_lo = 1
# upper bound
σδsq_hi = 10
# inital guess
σδsq_0 = 0.01

## variance of noise
# lower bound
σϵsq_lo = 0.01
# upper bound
σϵsq_hi = 2
# inital guess
σϵsq_0 = 2

## concatenate bounds and inital guesses
# bounds
bds = [(σδsq_lo, σδsq_hi),(σϵsq_lo, σϵsq_hi)]
# inital guesses
σsq_0 = [(σδsq_0, σϵsq_0)]

## Optimization

In [37]:
p = 2
X = np.tile(x, (1,p))
print(np.shape(X))

Σ_δ = sq_exp(X[:,0], σδsq_0)
Σ_ϵ = σϵsq_0*np.identity(len(X[:,0]))
Σ = Σ_δ + Σ_ϵ

θ = gls_est(X, y, Σ)

(5, 2)


ValueError: solve: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m,n)->(m,n) (size 2 is different from 5)